# Post Call Analytics

Welcome to this training module on post-call analytics use cases using Amazon SageMaker JumpStart. 

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience. 

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

- [수정필요] Choice of LLM models in Amazon SageMaker JumpStart
- One model handling multiple PCA tasks
- Handling long call transcripts

## Step 0. Install packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../utils')
sys.path.append('../templates')

In [3]:
install_needed = True  # should only be True once

In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U langchain

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## Step 1. Prepare Large Language Model (LLM)

In [4]:
import boto3
from termcolor import colored
from sagemaker.session import Session
from langchain.llms import AmazonAPIGateway
from lib_en import Llama2ContentHandlerAmazonAPIGateway, FalconContentHandlerEndpoint, FalconContentHandlerAmazonAPIGateway

In [5]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

In [6]:
MODEL_NAME = "FALCON-40B" #LLAMA2-7B 

In [7]:
RESTAPI_ID = "6bk4r5mo4f"
URL = f'https://{RESTAPI_ID}.execute-api.{aws_region}.amazonaws.com/api/'.replace('"','')
LLM_INFO = {
    "FALCON-40B": f"{URL}llm/falcon_40b",
    "LLAMA2-7B": f"{URL}llm/llama2_7b",
}
LLM_URL = LLM_INFO[MODEL_NAME]
HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

print (f'MODEL_NAME: {MODEL_NAME}\nLLM_URL: {LLM_URL}')

MODEL_NAME: FALCON-40B
LLM_URL: https://6bk4r5mo4f.execute-api.us-east-1.amazonaws.com/api/llm/falcon_40b


In [8]:
llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)

if MODEL_NAME == "FALCON-40B": llm.content_handler = FalconContentHandlerAmazonAPIGateway()
elif MODEL_NAME == "LLAMA2-7B": llm.content_handler = Llama2ContentHandlerAmazonAPIGateway()

## Step 2. Load transcript files

In [49]:
transcript_files = [
    "./call_transcripts/negative-refund.txt",
    "./call_transcripts/neutral-short.txt",
    "./call_transcripts/positive-partial-refund.txt",
    "./call_transcripts/aws-short.txt",
    "./call_transcripts/aws.txt"
]
transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())

In [50]:
for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

transcript #1: timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry



transcript #2: timestamp: 2023-01-28 08:26:49.219717

Customer: Hi, I'd like to check the balance on my account.

Retail Support: Sure thing! Can I have your account number or phone number associated with the account?

Customer: My phone number is (123) 456-7890.

Retail Support: Great, thank you. Let me pull up y



transcript #3: timestamp: 2022-12-28 08:26:49.219717

Agent: Thank you for calling [Retailer], my name is [Agent Name]. How may I assist you today?

Customer: Hi, I wanted to check on the status of my order. It was supposed to arrive today, but I haven't received it yet.

Agent: I'm sorry to hear that. Can I have 



transcript #4: What is AWS? AWS or Amazon W

## Step 3. Post Call Analysis

In [40]:
from langchain import PromptTemplate

### Step 3.1. Prompt Template
In this notebook, we'll be performing four different analyses(**Summary, Sentiment, Intent and Resolution**), and we'll need a template for each one. 

* Summary template

In [41]:
summary_template = """
Analyze the retail support call transcript below. Provide a detail summary of the conversation in complete sentence:

context: {transcript}

summary:"""

* Sentiment template

In [42]:
sentiment_template = """
This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL", "NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class:

context: {transcript}

sentiment: """

* intent template

In [44]:
intent_template = """
This is a intent classification program. What is the purpose of the customer call using following classes
["SHIPMENT_DELAY", "COMPLAIN_PRODUCT_DEFECT", "ACCOUNT_QUESTION"]. classify the conversation into one and exact one of these classes.
If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class. 

context: {transcript}

intent: """

### Step 3.2. Analysis

In [15]:
def analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    analysis_prompt = prompt.format(transcript=transcript)
    llm.model_kwargs = params

    print (colored(analysis_prompt, 'green'))

    response = llm(analysis_prompt)

    return response

In [20]:
PARAMS = {
    "FALCON-40B": {
        "max_new_tokens": 200,
        "max_length": 1024,
        "top_p": 0.95,
        "do_sample": False,
        "temperature": 0.01,
        "return_full_text": False,
        "include_prompt_in_result": False
    },
    "LLAMA2-7B": {
        'max_new_tokens': 128,
        'top_p': 0.9,
        'temperature': 0.1,
        'return_full_text': False
    },
}

* Summary analysis

In [17]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS[MODEL_NAME],
    template=summary_template
)

print (res)


Analyze the retail support call transcript below. Provide a detail summary of the conversation in complete sentence:

context: timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry to hear that you received a defective product. Can you please let me know what the issue is?

Customer: The product I received is broken and unusable. I spent a lot of money on it, and now I can't even use it! This is unacceptable, and I demand a solution right now!

Agent: I completely understand your frustration, and I'm sorry for any inconvenience this has caused you. Can you please provide me with your order number so that I can look into this for you?

Customer: 2357894561

Agent: Thank you. I'm sorry to hear about the defective product you received. We can de

* Sentiment analysis

In [18]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS[MODEL_NAME],
    template=sentiment_template
)

print (res)


This is a sentiment analysis program. What is the customer sentiment using following classes 
["POSITIVE", "NEUTRAL", "NEGATIVE"]. classify the conversation into one and exact one of these classes. 
If you don't know or not sure, please use ["NEUTRAL"] class. Do not try to make up a class:

context: timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry to hear that you received a defective product. Can you please let me know what the issue is?

Customer: The product I received is broken and unusable. I spent a lot of money on it, and now I can't even use it! This is unacceptable, and I demand a solution right now!

Agent: I completely understand your frustration, and I'm sorry for any inconvenience this has caused you. Can you please provide 

In [57]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS[MODEL_NAME],
    template=intent_template
)

print (res)


This is a intent classification program. What is the purpose of the customer call using following classes
["SHIPMENT_DELAY", "COMPLAIN_PRODUCT_DEFECT", "ACCOUNT_QUESTION"]. classify the conversation into one and exact one of these classes.
If you don't know, please use ["UNKNOWN"] class. Do not try to make up a class. 

context: timestamp: 2022-12-27 08:26:49.219717

Agent: Thank you for calling our retail support line. My name is ABC. How can I assist you today?

Customer: Yes, I have received a defective product, and I am extremely angry about it! This is unacceptable, and I want it resolved immediately!

Agent: I'm sorry to hear that you received a defective product. Can you please let me know what the issue is?

Customer: The product I received is broken and unusable. I spent a lot of money on it, and now I can't even use it! This is unacceptable, and I demand a solution right now!

Agent: I completely understand your frustration, and I'm sorry for any inconvenience this has cause

## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM. 

In [45]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [54]:
def long_call_analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning
    
    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")
        summary_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True) # map_reduce, refine

        transcript = summary_chain.run(docs)

    analysis_prompt = prompt.format(transcript=transcript) 
    analysis = llm(analysis_prompt)

    return analysis

In [55]:
PARAMS = {
    "FALCON-40B": {
        "max_new_tokens": 1024,
        "max_length": 1024,
        "top_p": 0.95,
        "do_sample": False,
        "temperature": 0.2,
        "return_full_text": False,
        "include_prompt_in_result": False
    },
    "LLAMA2-7B": {
        'max_new_tokens': 128,
        'top_p': 0.9,
        'temperature': 0.1,
        'return_full_text': False
    },
}

In [56]:
%%time

res = long_call_analysis(
    llm=llm,
    transcript=transcripts[3],
    params=PARAMS[MODEL_NAME],
    template=summary_template
)

print (res)

Now we have 3 documents and the first one has 210 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"What is AWS? AWS or Amazon Web Services is a cloud service provider that offers various computing services that are accessible over the public internet.

AWS and other public cloud vendors — like Google Cloud Platform (GCP) and Microsoft Azure — manage and maintain hardware and infrastructure, saving organizations and individuals the cost and complexity of purchasing and running resources on site. These resources may be accessed for free or on a pay-per-use basis.

To get a better understanding of AWS, it can help to wrap your head around how massive AWS is. There's no denying it, AWS is kind of a big deal. How big?

One in three sites you visit on the internet uses AWS services. 
In 2019, Amazon Web Services raked in more than $35 billion in revenue. If AWS were its own company, 